## Fake News Classifier Using LSTM

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
## Drop NAN Values

df = df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
## Get the Independent Features

X = df.drop('label', axis=1)

In [9]:
## Get the Dependent Feature

y = df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.10.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
### Vocabulary size

voc_size = 5000

## OneHot Representation

In [17]:
messages = X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.stem.porter import PorterStemmer ## Stemming Purpose
ps = PorterStemmer()

In [31]:
re.sub('[^a-zA-Z]', ' ', messages['title'][0])

'House Dem Aide  We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It'

In [32]:
len(messages)

18285

In [41]:
messages = messages.reset_index()

In [42]:
### Data Preprocessing

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # Remove Special charecters
    review = review.lower()
    review = review.split()  # Covert into list of ssentences
    entences list
    
    # Stemming and removing the stopwords
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [43]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [44]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [45]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3242, 1012, 467, 2118, 4691, 4981, 1674, 1943, 1762, 2996],
 [4858, 1623, 3276, 3333, 3176, 688, 1501],
 [4726, 2423, 1315, 2245],
 [3556, 1618, 1962, 1178, 3890, 4382],
 [1324, 3176, 31, 3153, 430, 3527, 3176, 3208, 3767, 4699],
 [2869,
  250,
  4708,
  3890,
  2661,
  363,
  2131,
  4282,
  1605,
  4581,
  336,
  3197,
  4474,
  2295,
  1501],
 [398, 1342, 4396, 67, 8, 4336, 3182, 2945, 322, 3059, 1415],
 [1878, 2807, 4793, 4002, 1968, 955, 363, 2253, 322, 3059, 1415],
 [365, 766, 1642, 193, 4018, 3984, 2879, 178, 363, 3574],
 [3971, 2521, 3692, 2026, 1460, 3951, 313, 3548],
 [3403, 2606, 4185, 3715, 4216, 4885, 1151, 4240, 3744, 4992, 2532],
 [1178, 575, 4691, 3984, 363, 1968],
 [3836, 1812, 1219, 2137, 4663, 3998, 3808, 2737, 4053],
 [2530, 3310, 2134, 107, 2963, 4907, 4024, 322, 3059, 1415],
 [2292, 3973, 1805, 481, 2493, 322, 3059, 1415],
 [2751, 4817, 1963, 2242, 2025, 1928, 4068, 3949, 3668, 158],
 [3194, 1933, 1623],
 [3509, 3435, 864, 478, 363, 2556, 2820, 1501],
 [395, 243

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr[1]

[4858, 1623, 3276, 3333, 3176, 688, 1501]

## Embedding Representation

In [50]:
sent_length = 20

embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen = sent_length)
print(embedded_docs)

[[3242 1012  467 ...    0    0    0]
 [4858 1623 3276 ...    0    0    0]
 [4726 2423 1315 ...    0    0    0]
 ...
 [3939 1137 2102 ...    0    0    0]
 [1167 1968  103 ...    0    0    0]
 [1022 2260 4937 ...    0    0    0]]


In [51]:
embedded_docs[0]

array([3242, 1012,  467, 2118, 4691, 4981, 1674, 1943, 1762, 2996,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [53]:
# Creating model

embedding_vector_features = 40 ## Feature Representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
len(embedded_docs), y.shape

(18285, (18285,))

In [55]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [56]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [59]:
### Finally Training

model.fit(X_train, y_train, validation_data=(X_test,y_test),epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 25ms/step - loss: 0.3182 - accuracy: 0.8445 - val_loss: 0.2006 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1346 - accuracy: 0.9506 - val_loss: 0.1917 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1001 - accuracy: 0.9665 - val_loss: 0.2215 - val_accuracy: 0.9162
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0652 - accuracy: 0.9780 - val_loss: 0.2435 - val_accuracy: 0.9133
Epoch 5/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0517 - accuracy: 0.9833 - val_loss: 0.2965 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0409 - accuracy: 0.9859 - val_loss: 0.3565 - val_accuracy: 0.9044
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0276 - accuracy: 0.9924 - val_loss: 0.3631 - val_accuracy: 0.9084

## Adding Dropout

In [69]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

## Performanace Metrics and Accuracy

In [63]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 4ms/step


In [65]:
y_pred = np.where(y_pred > 0.6, 1, 0)

In [66]:
from sklearn.metrics import confusion_matrix

In [67]:
confusion_matrix(y_test, y_pred)

array([[3100,  319],
       [ 245, 2371]], dtype=int64)

In [68]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9065451532725767

In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

